In [2]:
import pandas as pd
import os
import gc

from time import sleep

import gspread_dataframe as gs_df

os.chdir("../")

# INITIALIZE DRIVER
from selenium import webdriver
from sapiens import LoginPage, KEYS
import functions


% reload_ext autoreload
% autoreload 2

In [25]:
driver = webdriver.Firefox()
sapiens = LoginPage(driver).login('33831640807', 'Fer465023@ur_')

### Criar o Data Frame para exportar os dados

In [34]:
df = pd.DataFrame(columns=KEYS)

#planilha = pd.read_excel("files/planilha_obito.xlsx", sheet_name="consulta_RF", dtype=str)

In [27]:
planilha.columns = ["Nome", "cpf", "fistel", "situação", "estado"]

In [28]:
planilha = planilha.sort_values(by="Nome")
planilha.head()

,Nome,cpf,fistel,situação,estado
6779,CARLOS AUGUSTO RODRIGUES,00096537752,80104719923,nan,nan
3308,IFC INTERNATIONAL FOOD COMPANY INDUSTRIA DE A...,02473696000107,50403232201,nan,nan
2970,.JOSE ANTONIO SOARES,07362475802,80100864449,nan,nan
3427,11500 EMPREENDIMENTOS LTDA - EPP,04720254000135,50402126742,nan,nan
7092,A C A SANTOS-ME,64159692000106,50406300933,nan,nan


### Leitura dos Dados Brutos na Planilha

In [5]:
gc = functions.authenticate_google("files/anatel.json")

sht = gc.open(title='RC')

In [6]:
registros = sht.worksheet('Registros')
registros_rf = sht.worksheet('Registros_RF')

In [29]:
#lista_CPF = registros.col_values(1)[1:]

lista_cpf = planilha["cpf"]

In [39]:
df.shape

(1479, 16)

In [ ]:
counter = 0

for cpf in lista_cpf[1479:]:
    
    while len(cpf) < 11:
        
        cpf = '0' + str(cpf)
    
    registro = sapiens.get_registro(cpf)    
   
    if not registro:
    
        if len(cpf) == 11:
            
            sapiens.pesquisa_dados(cpf)            
            
        else:
            
            sapiens.registros[cpf] = None
            
            
        registro = sapiens.get_registro(cpf)
        
        
    print(registro)            
    
    df = df.append(pd.Series(registro), ignore_index=True)
    
    counter += 1
    
    if counter > 250:
        
        sapiens.close()
        
        del sapiens
        
        print(gc.collect())
        
        driver = webdriver.Firefox()
        
        sapiens = LoginPage(driver).login('33831640807', 'Fer465023@ur_')
        
        sleep(10)
        
        counter = 0  

{'CPF': '085.337.848-72', 'Número': '228', 'Sexo': 'MASCULINO', 'Ano do Óbito': '2004', 'Nome': 'CELSO GIOVANELLI SOARES', 'Logradouro': 'RUA PROFESSOR ARAUJO COELHO', 'Situação Cadastral': 'CANCELADA POR ÓBITO SEM ESPÓLIO', 'Data de Nascimento': '08-09-1932', 'Cidade': 'SAO PAULO/SP', 'Cep': '05138-000', 'Bairro': 'PIRITUBA', 'Título de Eleitor': '0000000000000', 'Complemento': 'JARDIM LIBANO', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'ANGELINA GIOVANELLI CASTANHO', 'Fone': '39023141'}
{'CPF': '045.751.638-05', 'Número': '201', 'Sexo': 'MASCULINO', 'Nome': 'CELSO JOSE TAVARES', 'Logradouro': 'RUA HERMINIA PASQUALETTI SERRA', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '10-11-1957', 'Cidade': 'SANTO ANDRE/SP', 'Cep': '09133-190', 'Bairro': 'SAO BERNARDO NOVO', 'Título de Eleitor': '0100734470124', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'MARIA JOSE MONTEIRO TAVARES', 'Fone': '98157352'}
{'CPF': '695.692.078-68', 'Número': '222', 'Sexo': 'MASCULINO', 'Nome': 'CELSO KUNIOSHI', 'Log

{'CPF': '193.669.143-49', 'Número': '216', 'Sexo': 'MASCULINO', 'Nome': 'CELSON FERNANDES CAVALCANTE', 'Logradouro': 'AVENIDA SAO FRANCISCO', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '11-02-1963', 'Cidade': 'NOVO ORIENTE/CE', 'Cep': '63740-000', 'Bairro': 'CENTRO', 'Título de Eleitor': '0287730670116', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'FRANCISCA TEREZINHA DE JESUS', 'Fone': '41815714'}
None
None
None
None
None
None
None
None
None
None
None
None
{'CPF': '004.467.688-39', 'Número': '11', 'Sexo': 'MASCULINO', 'Nome': 'CESAR AUGUSTO GOUVEA BATISTA', 'Logradouro': 'RUA SAMUEL LIBORIO DE AVILA', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '02-08-1963', 'Cidade': 'GUARULHOS/SP', 'Cep': '07135-250', 'Bairro': 'JD ADRIANA', 'Título de Eleitor': '0058998420108', 'Complemento': 'A', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'MARIA GOUVEIA', 'Fone': '74735805'}
{'CPF': '248.858.288-30', 'Número': '46', 'Sexo': 'MASCULINO', 'Nome': 'CESAR AUGUSTO MODESTO', 'Logradouro': 'R

{'CPF': '276.555.778-03', 'Número': '1650', 'Sexo': 'MASCULINO', 'Nome': 'CHRYSTIANO GARCIA', 'Logradouro': 'RUA JOAO MEDUETA', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '08-03-1978', 'Cidade': 'VINHEDO/SP', 'Cep': '13280-000', 'Bairro': 'CAPELA', 'Título de Eleitor': '0274201890159', 'Complemento': 'BLOCO C APTO 22', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'FRANCISCA JOANA DOS SANTOS GARCIA', 'Fone': '38862319'}
None
{'CPF': '178.509.008-98', 'Número': '51', 'Sexo': 'MASCULINO', 'Nome': 'CICERO ALVES DA SILVA', 'Logradouro': 'RUA DOS CRAVOS', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '20-02-1975', 'Cidade': 'SAO BERNARDO DO CAMPO/SP', 'Cep': '09783-001', 'Bairro': 'VL ESPERANCA', 'Título de Eleitor': '0243642990175', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'FRANCISCA MARIA DA SILVA', 'Fone': '43358539'}
{'CPF': '209.634.084-53', 'Número': '180', 'Sexo': 'MASCULINO', 'Nome': 'CICERO ARLINDO DUARTE', 'Logradouro': 'RUA MATO GROSSO', 'Situação Cadastral': 'REGULAR',

{'CPF': '142.173.798-11', 'Número': '115', 'Sexo': 'MASCULINO', 'Nome': 'CIRO GIRELLO RISKALLAH', 'Logradouro': 'OUTROS MIRO VETORAZZO', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '13-02-1962', 'Cidade': 'SAO BERNARDO DO CAMPO/SP', 'Cep': '09820-135', 'Bairro': 'DEMARCHI', 'Título de Eleitor': '0118494220183', 'Complemento': 'CASA 83', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'NILDE GIRELLO RISKALLAH', 'Fone': '99873364'}
{'CPF': '005.273.528-10', 'Número': '84', 'Sexo': 'MASCULINO', 'Nome': 'CIRO LUCIANO CARNEIRO FILHO', 'Logradouro': 'RUA JOSE S PONTO CARDOSO', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '28-07-1958', 'Cidade': 'PINDAMONHANGABA/SP', 'Cep': '12440-250', 'Bairro': 'MOREIRA CESAR', 'Título de Eleitor': '0214069650167', 'Complemento': 'CASA', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'MARIA APARECIDA DE LIMA CARNEIRO', 'Fone': '36371998'}
{'CPF': '095.760.628-10', 'Número': '1552', 'Sexo': 'MASCULINO', 'Ano do Óbito': '2005', 'Nome': 'CIRO MAURO DEGASPERI

{'CPF': '053.401.788-62', 'Número': '269', 'Sexo': 'MASCULINO', 'Nome': 'CLAUDEMIR RODRIGUES', 'Logradouro': 'RUA CAPIBERIBE', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '15-04-1962', 'Cidade': 'AMERICANA/SP', 'Cep': '13469-470', 'Bairro': 'VILA DAINESE', 'Título de Eleitor': '0054932430124', 'Complemento': 'APTO 612', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'RICARDINA PAIVA RODRIGUES', 'Fone': '30127123'}
{'CPF': '499.053.921-49', 'Número': '1-06', 'Sexo': 'MASCULINO', 'Ano do Óbito': '2005', 'Nome': 'CLAUDEMIR SABINO DE OLIVEIRA', 'Logradouro': 'RUA FRANCISCO ELIAS MACIERINHO', 'Situação Cadastral': 'CANCELADA POR ÓBITO SEM ESPÓLIO', 'Data de Nascimento': '27-11-1970', 'Cidade': 'BAURU/SP', 'Cep': '17000-000', 'Bairro': 'JD CHAPADAO', 'Título de Eleitor': '0000000000000', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'MARIA DE LOURDES SILVA OLIVEIRA', 'Fone': '00000000'}
{'CPF': '016.724.818-99', 'Número': '50', 'Sexo': 'MASCULINO', 'Nome': 'CLAUDEMIR VANDERLEI GONCALVES', 'Logrado

...

{'CPF': '050.360.899-87', 'Número': '4452', 'Sexo': 'MASCULINO', 'Nome': 'DANIEL DA SILVEIRA', 'Logradouro': 'OUTROS MONSENHOR GERCINO', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '19-12-1948', 'Cidade': 'JOINVILLE/SC', 'Cep': '89230-290', 'Bairro': 'JARIVATUBA', 'Título de Eleitor': '0012638260990', 'Complemento': 'CASA', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'HILDA SANT ANNA DA SILVEIRA', 'Fone': '33249800'}
{'CPF': '086.007.148-00', 'Número': '826', 'Sexo': 'MASCULINO', 'Nome': 'DANIEL DE JESUS FELINTO', 'Logradouro': 'RUA CELESTE ZENI CANTADOR', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '10-08-1966', 'Cidade': 'ARAUCARIA/PR', 'Cep': '83704-550', 'Bairro': 'FAZENDA VELHA', 'Título de Eleitor': '0122092240141', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'TERESINHA DE JESUS FELINTO', 'Fone': '36435570'}
{'CPF': '338.030.268-65', 'Número': '417', 'Sexo': 'MASCULINO', 'Nome': 'DANIEL DIAS', 'Logradouro': 'AVENIDA ARMANDO MITALO SETTI', 'Situação Cadastral': 'REGULAR'

{'CPF': '053.684.068-78', 'Número': '120', 'Sexo': 'MASCULINO', 'Nome': 'DANIEL RAIMUNDO DO ESPIRITO SANTO', 'Logradouro': 'RUA RESPLENDOR', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '12-11-1963', 'Cidade': 'GUARULHOS/SP', 'Cep': '07155-080', 'Bairro': 'CUMBICA', 'Título de Eleitor': '0113345860116', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'JOVINA RAIMUNDO', 'Fone': '64698615'}
{'CPF': '144.441.108-05', 'Número': '495', 'Sexo': 'MASCULINO', 'Nome': 'DANIEL RICARDO POLACHINI', 'Logradouro': 'RUA MARIA ADAME PATTARO', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '23-09-1972', 'Cidade': 'CAMPINAS/SP', 'Cep': '13084-280', 'Bairro': 'MODESTO FERNANDES', 'Título de Eleitor': '0200337490116', 'Complemento': 'CASA 32', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'MARIA ZELIA FULACHI POLACHINI', 'Fone': '33252944'}
{'CPF': '283.092.158-51', 'Número': '16', 'Sexo': 'MASCULINO', 'Nome': 'DANIEL SECCO ZANOTTO', 'Logradouro': 'RUA ARACATUBA', 'Situação Cadastral': 'REGULAR', 'Data de

{'CPF': '319.018.808-42', 'Número': '1327', 'Sexo': 'MASCULINO', 'Nome': 'DANILO SINHEI IHA', 'Logradouro': 'RUA SAO JOAO', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '09-08-1983', 'Cidade': 'JABOTICABAL/SP', 'Cep': '14882-010', 'Bairro': 'APARECIDA', 'Título de Eleitor': '0309480840116', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'CELIA LAGUARDIA IHA', 'Fone': '32023544'}
{'CPF': '122.737.288-40', 'Número': '101', 'Sexo': 'MASCULINO', 'Nome': 'DARCI DE SOUZA NUNES', 'Logradouro': 'RUA ISIS DE CAMARGO BARROS MARTINS', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '26-04-1972', 'Cidade': 'SOROCABA/SP', 'Cep': '18077-330', 'Bairro': 'JARDIM SANTO ANDRE', 'Título de Eleitor': '0273093950191', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'LUZIA FERREIRA NUNES', 'Fone': '97052132'}
{'CPF': '173.036.908-16', 'Número': '110', 'Sexo': 'MASCULINO', 'Nome': 'DARCI JUVINIANO DA SILVA', 'Logradouro': 'RUA LAURA', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '06-12-1961', 'Cidade

{'CPF': '180.330.558-47', 'Número': '197', 'Sexo': 'MASCULINO', 'Nome': 'DAVID ALVARO DA SILVA', 'Logradouro': 'RUA JAIME DUARTE', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '04-05-1976', 'Cidade': 'CUBATAO/SP', 'Cep': '11534-040', 'Bairro': 'JD NOVA REPUBLICA', 'Título de Eleitor': '0238927620183', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'EUNICE SILVA', 'Fone': '78506205'}
{'CPF': '215.687.328-30', 'Número': '543', 'Sexo': 'MASCULINO', 'Nome': 'DAVID CORREIA RODRIGUES SILVA', 'Logradouro': 'AVENIDA ENG FRANCISCO PRESTES MAIA', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '23-09-1979', 'Cidade': 'SAO JOSE DOS CAMPOS/SP', 'Cep': '12240-680', 'Bairro': 'JD DAS INDUSTRIAS', 'Título de Eleitor': '0286848180108', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'SUELY RODRIGUES SILVA', 'Fone': '00000000'}
{'CPF': '308.926.458-73', 'Número': '62', 'Sexo': 'MASCULINO', 'Nome': 'DAVID CRISTOFHOR DE LIMA', 'Logradouro': 'RUA AGNALDO MACEDO', 'Situação Cadastral': 'REGULAR', 'Data de Na

{'CPF': '214.005.168-89', 'Número': '30', 'Sexo': 'MASCULINO', 'Nome': 'DEMETRIUS DE ANDRADE NASCIMENTO', 'Logradouro': 'RUA IBITIGUACU', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '01-08-1977', 'Cidade': 'SAO PAULO/SP', 'Cep': '03578-180', 'Bairro': 'JARDIM ELIANE', 'Título de Eleitor': '0243858470175', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'IRACEMA DE ANDRADE NASCIMENTO', 'Fone': '22530418'}
{'CPF': '214.005.168-89', 'Número': '30', 'Sexo': 'MASCULINO', 'Nome': 'DEMETRIUS DE ANDRADE NASCIMENTO', 'Logradouro': 'RUA IBITIGUACU', 'Situação Cadastral': 'REGULAR', 'Data de Nascimento': '01-08-1977', 'Cidade': 'SAO PAULO/SP', 'Cep': '03578-180', 'Bairro': 'JARDIM ELIANE', 'Título de Eleitor': '0243858470175', 'Nacionalidade': 'BRASILEIRO', 'Mãe': 'IRACEMA DE ANDRADE NASCIMENTO', 'Fone': '22530418'}
{'CPF': '155.419.058-42', 'Número': '1286', 'Sexo': 'MASCULINO', 'Nome': 'DEMETRIUS FERNANDO LUCIO', 'Logradouro': 'RUA BERTIOGA', 'Situação Cadastral': 'REGULAR', 'Data de Nasciment

In [ ]:
df = df.fillna("")

In [ ]:
cols = ["Logradouro", "Número", "Complemento", "Bairro", "Cep", "Cidade"]

df["Endereço_Completo"] = df.apply(lambda row: " ".join([str(row["Logradouro"]), 
                                                         str(row["Número"]), 
                                                         str(row["Complemento"]),
                                                         str(row["Bairro"]), "\n", "CEP:", 
                                                         str(row["Cep"]), 
                                                         str(row["Cidade"])]
                                                       ), axis=1
                                  )

In [24]:
df.to_excel("registros_cadin.csv", index=False)

In [25]:
#gs_df.set_with_dataframe(registros_rf, df, row=142, include_column_header=False)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Nome"]), row=2, col=9, include_column_header=False)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Endereço_Completo"]), row=2, col=12, include_column_header=False)


APIError: {
  "error": {
    "code": 401,
    "message": "Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",
    "status": "UNAUTHENTICATED"
  }
}


In [27]:
registros.row_count

2006